<a href="https://colab.research.google.com/github/rlagusdn04/XRPL/blob/main/notebooks/colab-github-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Google Colab with GitHub




[Google Colaboratory](http://colab.research.google.com) is designed to integrate cleanly with GitHub, allowing both loading notebooks from github and saving notebooks to github.

## Loading Public Notebooks Directly from GitHub

Colab can load public github notebooks directly, with no required authorization step.

For example, consider the notebook at this address: https://github.com/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb.

The direct colab link to this notebook is: https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb.

To generate such links in one click, you can use the [Open in Colab](https://chrome.google.com/webstore/detail/open-in-colab/iogfkhleblhcpcekbiedikdehleodpjo) Chrome extension.

## Browsing GitHub Repositories from Colab

Colab also supports special URLs that link directly to a GitHub browser for any user/organization, repository, or branch. For example:

- http://colab.research.google.com/github will give you a general github browser, where you can search for any github organization or username.
- http://colab.research.google.com/github/googlecolab/ will open the repository browser for the ``googlecolab`` organization. Replace ``googlecolab`` with any other github org or user to see their repositories.
- http://colab.research.google.com/github/googlecolab/colabtools/ will let you browse the main branch of the ``colabtools`` repository within the ``googlecolab`` organization. Substitute any user/org and repository to see its contents.
- http://colab.research.google.com/github/googlecolab/colabtools/blob/main will let you browse ``main`` branch of the ``colabtools`` repository within the ``googlecolab`` organization. (don't forget the ``blob`` here!) You can specify any valid branch for any valid repository.

## Loading Private Notebooks

Loading a notebook from a private GitHub repository is possible, but requires an additional step to allow Colab to access your files.
Do the following:

1. Navigate to http://colab.research.google.com/github.
2. Click the "Include Private Repos" checkbox.
3. In the popup window, sign-in to your Github account and authorize Colab to read the private files.
4. Your private repositories and notebooks will now be available via the github navigation pane.

## Saving Notebooks To GitHub or Drive

Any time you open a GitHub hosted notebook in Colab, it opens a new editable view of the notebook. You can run and modify the notebook without worrying about overwriting the source.

If you would like to save your changes from within Colab, you can use the File menu to save the modified notebook either to Google Drive or back to GitHub. Choose **File→Save a copy in Drive** or **File→Save a copy to GitHub** and follow the resulting prompts. To save a Colab notebook to GitHub requires giving Colab permission to push the commit to your repository.

## Open In Colab Badge

Anybody can open a copy of any github-hosted notebook within Colab. To make it easier to give people access to live views of GitHub-hosted notebooks,
colab provides a [shields.io](http://shields.io/)-style badge, which appears as follows:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)

The markdown for the above badge is the following:

```markdown
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
```

The HTML equivalent is:

```HTML
<a href="https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
```

Remember to replace the notebook URL in this template with the notebook you want to link to.

In [ ]:
#!git clone https://github.com/rlagusdn04/XRPL

import os
os.chdir("/content")
!pwd

from google.colab import drive
drive.mount('/content/drive')



/content
Mounted at /content/drive


In [12]:
# nest_asyncio와 xrpl-py 설치
!pip install nest_asyncio xrpl-py

# Colab 환경의 asyncio event loop 문제 해결
#AsyncIO: 비동기 프로그래밍을 위한 모듈, CPU작업과 IO작업을 병렬 처리
import nest_asyncio
nest_asyncio.apply() # 병렬 처리 시 중첩된 event loop 사용을 허용

# 모듈 임포트
from xrpl.clients import JsonRpcClient
from xrpl.wallet import generate_faucet_wallet
from xrpl.account import get_balance
from xrpl.models.requests import ServerInfo
from xrpl.models.transactions import NFTokenMint
from xrpl.transaction import submit_and_wait
import xrpl.utils

def register_nft_with_root_network(nft_details: dict):
    """
    NFT를 가상 API(The Root Network)에 등록하는 함수.
    실제 API 요청은 해당 문서를 참조하여 구현해야 함.
    """
    print("NFT가 The Root Network에 등록되었습니다:", nft_details)

def main():
    # XRPL 테스트넷에 연결
    testnet_url = "https://s.altnet.rippletest.net:51234/"
    client = JsonRpcClient(testnet_url)

    # 서버 정보 확인
    server_info = client.request(ServerInfo())
    print("서버 정보:", server_info.result)

    # 새로운 테스트넷 지갑 생성 (Faucet 이용)
    wallet = generate_faucet_wallet(client)
    print(f"지갑 주소: {wallet.classic_address}")
    print(f"비밀키: {wallet.seed}")

    # 잔액 확인
    balance = get_balance(wallet.classic_address, client)
    print(f"잔액: {balance} XRP")

    # NFT 민팅 트랜잭션 생성 (Tokenization)
    nft_mint_tx = NFTokenMint(
        account=wallet.classic_address,
        nftoken_taxon=0,  # NFT 분류 (0: 기본 분류)
        uri=xrpl.utils.str_to_hex("https://example.com/nft/metadata.json"),
        flags=8  # 8: Transferable(양도가능) NFT 설정
    )
    print("NFT 민팅 중...")
    nft_response = submit_and_wait(nft_mint_tx, client, wallet)
    print("NFT 민팅 결과:", nft_response.result)

    # NFT 민팅 결과가 성공적이면, The Root Network에 등록
    if "hash" in nft_response.result:
        nft_hash = nft_response.result["hash"]
        nft_details = {
            "nft_hash": nft_hash,
            "metadata_uri": "https://example.com/nft/metadata.json",
            "owner": wallet.classic_address,
        }
        register_nft_with_root_network(nft_details)
    else:
        print("NFT 등록 실패")

if __name__ == "__main__":
    main()



서버 정보: {'info': {'build_version': '2.4.0', 'complete_ledgers': '6-5531244', 'hostid': 'TUN', 'initial_sync_duration_us': '33813009', 'io_latency_ms': 1, 'jq_trans_overflow': '0', 'last_close': {'converge_time_s': 2.004, 'proposers': 6}, 'load_factor': 1, 'network_id': 1, 'peer_disconnects': '39392', 'peer_disconnects_resources': '527', 'peers': 101, 'ports': [{'port': '2459', 'protocol': ['peer']}, {'port': '51233', 'protocol': ['ws2', 'wss2']}, {'port': '50051', 'protocol': ['grpc']}], 'pubkey_node': 'n9KEk3TLMuoiTTLgrWWmfYm99hHFBZTXWzoyrHr3FbJWmRCXm96v', 'server_state': 'full', 'server_state_duration_us': '375663267875', 'state_accounting': {'connected': {'duration_us': '12003799', 'transitions': '1'}, 'disconnected': {'duration_us': '16803301', 'transitions': '1'}, 'full': {'duration_us': '375663267875', 'transitions': '1'}, 'syncing': {'duration_us': '5005871', 'transitions': '1'}, 'tracking': {'duration_us': '36', 'transitions': '1'}}, 'time': '2025-Mar-11 08:01:17.311130 UTC', 'u